In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 8)
!date

%load_ext autoreload
%autoreload 2

Sat Apr  1 12:40:12 PDT 2023


# Using dask on the IHME slurm cluster to load a large, distributed dataset

# set up a dask cluster

In [2]:
import dask_jobqueue

In [3]:
dask_jobqueue.__version__

'0.8.1+3.gaf044b4'

In [4]:
cluster = dask_jobqueue.SLURMCluster(
     cores=1,
     memory='32GB',
     processes=1,
#      local_directory='$TMPDIR',
     queue='all.q',
     account='proj_simscience',
     walltime='05:00:00',
)


/homes/abie/.conda/envs/bambi_env/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40373 instead
  warnings.warn(


In [5]:
cluster.scale(jobs=500)

# Now set up a dask client to use the cluster

In [6]:
import dask.distributed
client = dask.distributed.Client(cluster)

# Now use the dask dataframe over the distributed dask cluster

In [7]:
dname = '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/full_scale_334mil/united_states_of_america/2023_03_30_10_23_13/final_results/2023_03_30_16_02_39'

In [8]:
!ls -halt $dname/decennial_census_observer/ |wc -l

337


In [9]:
# https://docs.dask.org/en/stable/dataframe.html

import dask.dataframe as dd

In [10]:
df = dd.read_csv(f'{dname}/tax_w2_observer/tax_w2_observer_*.csv.bz2', dtype='category', blocksize=None)

In [11]:
%%time

len(df) / 1e9 # billions of records, takes about 1 hr to load and count??

CPU times: user 6min 42s, sys: 36.7 s, total: 7min 19s
Wall time: 39min 28s


3.258616466

In [12]:
%%time

df.head()  # takes about 7 minutes to get the first five rows, is this loading all files?

CPU times: user 1min 9s, sys: 7.59 s, total: 1min 16s
Wall time: 6min 36s


,simulant_id,employer_city,income,first_name,employer_id,mailing_address_unit_number,employer_street_name,mailing_address_street_name,date_of_birth,mailing_address_po_box,...,mailing_address_zipcode,tax_form,mailing_address_street_number,ssn,middle_initial,mailing_address_city,employer_unit_number,employer_street_number,employer_name,employer_state
0,1007_0,annville,113686.12887817506,Aida,352747,NaN,edgecliff ct,pleasant street,1956-04-15,0,...,46368,W2,5917,212-23-6302,G,indianapolis,NaN,NaN,Kipp Halifax Health & K 4 All Race Place,PA
1,1007_1,sherwood,19497.552528920678,Alan,1471774,NaN,n 52nd st,lewis,1951-05-02,0,...,32725,W2,532,832-25-7431,H,st. petersburg,NaN,309,Norman Rockwell Area Fire District 8,OR
2,1007_1,cottonwood heights,63576.776206371454,Alan,1681333,NaN,skyview ter,lewis,1951-05-02,0,...,32725,W2,532,832-25-7431,H,st. petersburg,unit 170,1960,Roadrunners Head Brewing Company,UT
3,1007_3,nashville,21680.05131244091,Adriana,550091,NaN,ince dr,f st,1975-12-07,0,...,06413,W2,26264,600-73-2024,H,new canaan,NaN,e,Master Rick Auto Body Works,TN
4,1007_3,schiller park,30202.48838663811,Adriana,933603,NaN,stoney crk cir,f st,1975-12-07,0,...,06413,W2,26264,600-73-2024,H,new canaan,NaN,NaN,Marshall County Volunteer Fire Department Stat...,IL


In [13]:
%%time

employer_size_dist = df.employer_name.value_counts().compute()  # took 40 minutes?
employer_size_dist

CPU times: user 2min 26s, sys: 11.8 s, total: 2min 38s
Wall time: 13min 58s


Military                                45145499
Seattle Branch Library                    173451
Lisa's Leaves of Long Park                155627
Crossing Pet Grooming                     119698
                                          ...   
Rogers MA                                      6
Visiting Hearts At the Box Food Bank           5
Ninth Street Dough                             5
Hatfield Roofing Supply                        3
Name: employer_name, Length: 1571136, dtype: int64

In [14]:
%%time

alt_employer_size_dist = df.employer_id.value_counts().compute()  # will this one work?
alt_employer_size_dist

CPU times: user 4min 33s, sys: 24.9 s, total: 4min 58s
Wall time: 26min 19s


1          45139497
488652       173451
1628100      155627
929500       119698
             ...   
96103             6
1084015           5
1424013           5
1738115           3
Name: employer_id, Length: 1745101, dtype: int64

In [15]:
%%time
# this takes around 70 minutes

t = df.groupby('employer_id').employer_name.nunique().compute()
t

CPU times: user 17min 24s, sys: 1min 46s, total: 19min 11s
Wall time: 1h 43min 15s


employer_id
1          1
1000080    1
1000112    1
1000201    1
          ..
444780     1
854654     1
96103      1
1424013    1
Name: employer_name, Length: 1745101, dtype: int64

In [16]:
t.value_counts()

1    1745100
0          1
Name: employer_name, dtype: int64

In [17]:
# I wonder how the employer_id with no employer name got in there, and which it is?

# TODO: compare employer_size_dist, alt_employer_size_dist to confirm that unemployed has an id, but not a name